In [ ]:
from model import DFL_VGG16,DFL_RESNET,DFL_EfficientNet,MangoNet
from dataset import MangoDataset
from sklearn.metrics import confusion_matrix
from util import mixup_data, mixup_criterion, plot_confusion_matrix
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline

## load data

In [ ]:
BATCH_SIZE=8
trainset = MangoDataset(root = './', train = "train")
valset = MangoDataset(root = './', train = "val")
testset = MangoDataset(root = './', train = "test")
# create train/val loaders
train_loader = DataLoader(dataset=trainset,
                          batch_size=BATCH_SIZE, 
                          shuffle=True,
                          num_workers=1)
train_n = len(trainset)
val_n = len(valset)
val_loader = DataLoader(dataset=valset,
                        batch_size=BATCH_SIZE, 
                        shuffle=False,
                        num_workers=1)

test_loader = DataLoader(dataset=testset,
                        batch_size=BATCH_SIZE, 
                        shuffle=False,
                        num_workers=1)

## DFL 

In [ ]:
EPOCH = 50
DEVICE = "cuda:1"
# model = DFL_EfficientNet().to(DEVICE)
# model = DFL_RESNET().to(DEVICE)
model = DFL_VGG16().to(DEVICE)
model.load_state_dict(torch.load("DFL_VGG16"))
ce = torch.nn.NLLLoss()
# opt = torch.optim.Adam([{'params': model.parameters(), 'lr': 0.0001}]) ## for efficientnet
opt = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay = 0.000005)
for i in range(EPOCH):
    model.train()
    for data, label in tqdm(train_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1, out2, out3 = model(data)
        out1, out2, out3 = F.log_softmax(out1,1), F.log_softmax(out2,1), F.log_softmax(out3,1)
        loss = ce(out1,label)+ce(out2,label)+0.1*ce(out3,label)
        loss.backward()
        opt.step()
        opt.zero_grad()
    print(loss)
    torch.cuda.empty_cache()
    model.eval()
    correct = 0
    for data, label in tqdm(val_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1, out2, out3 = model(data)
        out = out1+out2+0.1*out3
        pred = torch.max(out,axis=1)[1]
        correct += torch.sum((label==pred).int()).detach().item()
    print("valacc:",correct/val_n)
    torch.cuda.empty_cache()
    model.eval()
    correct = 0
    for data, label in tqdm(train_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1, out2, out3 = model(data)
        out = out1+out2+0.1*out3
        pred = torch.max(out,axis=1)[1]
        correct += torch.sum((label==pred).int()).detach().item()
    print("trainacc:",correct/train_n)
    torch.cuda.empty_cache()
#     torch.save(model.state_dict(), "DFL_VGG16_1")

## EfficientNet + Mixup

In [ ]:

EPOCH = 50
DEVICE = "cuda:1"
model = MangoNet().to(DEVICE)

ce = torch.nn.NLLLoss()
opt = torch.optim.Adam([{'params': model.parameters(), 'lr': 0.0001}])
# opt = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay = 0.000005)
for i in range(EPOCH):
    model.train()
    for data, label in tqdm(train_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1 = model(data)
        out1 = F.log_softmax(out1,1)
        loss = ce(out1,label)
        mixed_x, y_a, y_b, lam = mixup_data(data, label)
        pred = model(mixed_x)
        mixloss = mixup_criterion(ce, pred, y_a, y_b, lam)
        closs = loss+mixloss
        closs.backward()
        opt.step()
        opt.zero_grad()
    print(closs)
    model.eval()
    correct = 0
    for data, label in tqdm(val_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1 = model(data)

        pred = torch.max(out1,axis=1)[1]
        correct += torch.sum((label==pred).int()).detach().item()
    print("valacc:",correct/val_n)
    model.eval()
    correct = 0
    for data, label in tqdm(train_loader):
        data, label = data.to(DEVICE), label.to(DEVICE)
        out1 = model(data)
        pred = torch.max(out1,axis=1)[1]
        correct += torch.sum((label==pred).int()).detach().item()
    print("trainacc:",correct/train_n)
    torch.save(model.state_dict(), "model")
#     

## DFL Feature Visualization

In [ ]:

DEVICE = "cuda:1"
model = DFL_VGG16().to(DEVICE)
model.load_state_dict(torch.load("DFL_VGG16"))
model.eval()
correct = 0
preds,labels,val_label,val_pred=[],[],[],[]
test_feature_a,train_feature_a,val_feature_a=[],[],[]
for data in tqdm(test_loader):
    data = data.to(DEVICE)
    out1,out2,out3,feature_a,feature_b = model(data)
    test_feature_a.append(feature_b.data.cpu())
    out = out1+out2+0.1*out3
    pred = torch.max(out,axis=1)[1]
    preds.append(pred)
for data,label in tqdm(val_loader):
    data,label = data.to(DEVICE),label.to(DEVICE)
    out1,out2,out3,feature_a,feature_b = model(data)
    val_feature_a.append(feature_b.data.cpu())
    val_label.append(label.cpu())
for data,label in tqdm(train_loader):
    data,label = data.to(DEVICE),label.to(DEVICE)
    out1,out2,out3,feature_a,feature_b = model(data)
    train_feature_a.append(feature_b.data.cpu())
    labels.append(label.cpu())
test_feature_a = np.asarray(torch.cat(test_feature_a).cpu().numpy())
train_feature_a = np.asarray(torch.cat(train_feature_a).cpu().numpy())
val_feature_a = np.asarray(torch.cat(val_feature_a).cpu().numpy())
labels = np.asarray(torch.cat(labels).cpu().numpy())
val_label = np.asarray(torch.cat(val_label).cpu().numpy())
# preds = np.asarray(torch.cat(preds).cpu().numpy())
#     correct += torch.sum((label==pred).int()).detach().item()
# print("trainacc:",correct/val_n)
n=300
a = np.concatenate((train_feature_a[:n].reshape(n,-1),test_feature_a[:n].reshape(n,-1)),axis=0)
a = np.concatenate((a,val_feature_a[:n].reshape(n,-1)),axis=0)
X_embedded = TSNE(n_components=2).fit_transform(a)
plt.scatter(X_embedded[:n,0],X_embedded[:n,1],c=labels[:n],label="train",marker=".",s=50)
plt.scatter(X_embedded[n:2*n,0],X_embedded[n:2*n,1],c="black",label="test",marker=".",s=50)
plt.scatter(X_embedded[2*n:,0],X_embedded[2*n:,1],c=val_label[:n],label="val",marker=".",s=50)
plt.legend(loc='upper right')
plt.show()

## EfficientNet Feature Visualization

In [ ]:
DEVICE = "cuda:1"

class sep_effiecient(nn.Module):
    def __init__(self):
        super(sep_effiecient, self).__init__()
        self.model = MangoNet().to(DEVICE)
        self.model.load_state_dict(torch.load("model"))
    def forward(self, x):
        f = self.model.backbone.extract_features(x)
        return f
correct = 0
preds,labels,val_label,val_pred=[],[],[],[]
test_feature_a,train_feature_a,val_feature_a=[],[],[]
model = sep_effiecient().to(DEVICE)
for data in tqdm(test_loader):
    data = data.to(DEVICE)
    feature_b = model(data)
    test_feature_a.append(feature_b.data.cpu())
#     pred = torch.max(out,axis=1)[1]
#     preds.append(pred)
for data,label in tqdm(val_loader):
    data,label = data.to(DEVICE),label.to(DEVICE)
    feature_b = model(data)
    val_feature_a.append(feature_b.data.cpu())
    val_label.append(label.cpu())
for data,label in tqdm(train_loader):
    data,label = data.to(DEVICE),label.to(DEVICE)
    feature_b = model(data)
    train_feature_a.append(feature_b.data.cpu())
    labels.append(label.cpu())
test_feature_a = np.asarray(torch.cat(test_feature_a).cpu().numpy())
train_feature_a = np.asarray(torch.cat(train_feature_a).cpu().numpy())
val_feature_a = np.asarray(torch.cat(val_feature_a).cpu().numpy())
labels = np.asarray(torch.cat(labels).cpu().numpy())
val_label = np.asarray(torch.cat(val_label).cpu().numpy())

train_feature_a = np.mean(train_feature_a,axis=(2,3))
test_feature_a = np.mean(test_feature_a,axis=(2,3))
val_feature_a = np.mean(val_feature_a,axis=(2,3))
n=300
a = np.concatenate((train_feature_a[:n].reshape(n,-1),test_feature_a[:n].reshape(n,-1)),axis=0)
a = np.concatenate((a,val_feature_a[:n].reshape(n,-1)),axis=0)
X_embedded = TSNE(n_components=2).fit_transform(a)
plt.scatter(X_embedded[:n,0],X_embedded[:n,1],c=labels[:n],label="train",marker=".",s=50)
plt.scatter(X_embedded[n:2*n,0],X_embedded[n:2*n,1],c="black",label="test",marker=".",s=50)
plt.scatter(X_embedded[2*n:,0],X_embedded[2*n:,1],c=val_label[:n],label="val",marker=".",s=50)
plt.legend(loc='upper right')
# plt.savefig(method+"scatter"+str(step)+".png")
plt.show()

## EfficientNet Testing

In [ ]:
DEVICE = "cuda:1"
model = MangoNet().to(DEVICE)
model.load_state_dict(torch.load("model"))
model.eval()
correct = 0
preds=[]
labels=[]
for data,label in tqdm(val_loader):
    data,label = data.to(DEVICE),label.to(DEVICE)
    out = model(data)
#     out = out1+out2+0.1*out3
    pred = torch.max(out,axis=1)[1]
    preds.append(pred)
    labels.append(label)
preds = np.asarray(torch.cat(preds).cpu().numpy())
labels = np.asarray(torch.cat(labels).cpu().numpy())

## Confusion Matrix

In [ ]:
model = DFL_VGG16().to(DEVICE)
model.load_state_dict(torch.load("DFL_VGG16"))
model.eval()
correct = 0
preds=[]
labels=[]
for data,label in tqdm(val_loader):
    data,label = data.to(DEVICE),label.to(DEVICE)
    out1,out2,out3,feature_a,feature_b = model(data)
    out = out1+out2+0.1*out3
    pred = torch.max(out,axis=1)[1]
    preds.append(pred)
    labels.append(label)
preds = np.asarray(torch.cat(preds).cpu().numpy())
labels = np.asarray(torch.cat(labels).cpu().numpy())
cnf_matrix = confusion_matrix(labels, preds)
plot_confusion_matrix(cnf_matrix,classes=[a for a in range(3)],normalize=True)
plt.show()

DEVICE = "cuda:0"
model = MangoNet().to(DEVICE)
model.load_state_dict(torch.load("model"))
model.eval()
correct = 0
preds,labels=[],[]
for data,label in tqdm(val_loader):
    data = data.to(DEVICE)
    out = model(data)
#     out = out1+out2+0.1*out3
    pred = torch.max(out,axis=1)[1]
    labels.append(label)
    preds.append(pred)
preds = np.asarray(torch.cat(preds).cpu().numpy())
labels = np.asarray(torch.cat(labels).cpu().numpy())
cnf_matrix = confusion_matrix(labels, preds)
plot_confusion_matrix(cnf_matrix,classes=[a for a in range(3)],normalize=True)
plt.show()

## Submit Result

In [ ]:

DEVICE = "cuda:1"
model = MangoNet().to(DEVICE)
model.load_state_dict(torch.load("model"))
model.eval()
correct = 0
preds=[]
for data in tqdm(test_loader):
    data = data.to(DEVICE)
    out = model(data)
#     out = out1+out2+0.1*out3
    pred = torch.max(out,axis=1)[1]
    preds.append(pred)
preds = np.asarray(torch.cat(preds).cpu().numpy())
preds = np.where(preds==0, "A", preds)
preds = np.where(preds=="1", "B", preds)
preds = np.where(preds=="2", "C", preds)
df = pd.read_csv("test_example.csv")
df["label"]=pd.Series(preds)
df = df.set_index("image_id")
df.to_csv("result_DFL.csv")